In [1]:
import pyximport; pyximport.install()
from csvparser import *
from gpcode import *

from functools import partial

In [2]:
csv_filename = "Distance Results/Most-distant-" + machine_alias + "-2018-02-05 02:14:03.642540.csv.txt"

fields = [['Heuristic'], ['Fitness']]
conditions = []

csv_data = extractDictFromCSV(csv_filename)
parsed_data = extractIndividualsFromAASData(csv_data, ind_field=fields[0], fitness_field=fields[1], condition=conditions)
parsed_data = sorted(parsed_data, reverse=True)

In [7]:
opponent_population = []

for p in parsed_data:
    opponent_population.append(gp.PrimitiveTree.from_string(p.heuristic, pset))
    print p

IfThenElse(and_(lowestCardLE(ARG0, 7), highestCardLE(ARG0, 11)), 'check', IfThenElse(totalPotLE(ARG1, 2), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', 'check'))) | 0 | 0.777696875
IfThenElse(and_(totalPotLE(ARG1, 7), totalPotGE(ARG1, 3)), 'check', IfThenElse(highestCardGE(ARG0, 12), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardGE(ARG0, 10), 'raise', IfThenElse(isSameSuit(ARG0), 'check', IfThenElse(totalPotGE(ARG1, 17), 'raise', 'check')))))) | 0 | 0.744421875
IfThenElse(and_(lowestCardLE(ARG0, 7), highestCardLE(ARG0, 11)), 'check', IfThenElse(totalPotLE(ARG1, 2), 'raise', IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(totalPotGE(ARG1, 6), 'fold', 'check')))) | 0 | 0.743465625
IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse(lowestCardGE(ARG0, 13), 'raise', IfThenElse(highestCardLE(ARG0, 11), 'check', IfThenElse(totalPotGE(ARG1, 6), 'raise', IfThenElse(totalPotLE(ARG1, 2), 'raise', 'check'))))) | 0 | 0.691153125
IfThenElse(hasDoubles(ARG0), 'raise', IfThenElse

In [4]:
def makeHeuristicPopulation(heuristic_list_filename):
    population = []
    heuristic_file = open(heuristic_list_filename, 'r')
    
    for line in heuristic_file:
        if len(line) > 3:
            #population.append(gp.PrimitiveTree.from_string(line, pset))
            population.append(line)
    
    heuristic_file.close()
    
    return population

In [5]:
def findFitnessMatchUp(population, opponent_heuristic, n):
    pop = population
    
    pool = mp.Pool(mp.cpu_count())
    
    p2 = HeuristicAI()
    p2.setHeuristic(marshal.dumps(toolbox.compile(opponent_heuristic).func_code))
    
    temp = []
    for i in range(0, len(pop)):
        temp.append(HeuristicAI())
        temp[i].setHeuristic(marshal.dumps(toolbox.compile(pop[i]).func_code))
    fitness_list = pool.map(partial(toolbox.evaluate, opponent=p2), temp)
    pool.terminate()
    
    for i in range(0, len(fitness_list)):
        pop[i].fitness = fitness_list[i]
    
    return pop

In [6]:
num_sims = 80000 * 10

heuristic_list_filename = "Comparison Results/list_of_heuristics.txt"
population = makeHeuristicPopulation(heuristic_list_filename)

temp = []
for i in range(0, len(population)):
    heuristic = simplifyTree(simplifyByRemovingStatements(gp.PrimitiveTree.from_string(population[i], pset), all_hands), all_hands)
    if not isThereCopyOfIndividual(temp, heuristic):
        temp.append(heuristic)

population = temp

counter = 0
for opponent in opponent_population:
    toolbox.register("evaluate", evaluateMatchUp, n=num_sims)

    csv_filename = "Distance Results/Versus-" + str(counter) + "-" + machine_alias + ".csv.txt"
    createLogFile(csv_filename, "Num Sims; Heuristic; Complexity; Fitness;\n")

    population = findFitnessMatchUp(population, opponent, n=num_sims)
    for p in population:
        logToFile(csv_filename, str(num_sims) + "; " + str(p) + "; " + str(findComplexity(p)) + "; " + str(p.fitness[0]) + ";\n")

    counter += 1